In [1]:
## import general libraries
import numpy as np
import pandas as pd
import re
import json
import glob
# import nltk
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
# from gensim.models import coherencemodel
from APIs.Data_Cleaning import datacleaning, datamapping, datapreprocessing

import nltk
nltk.data.path.append("/home/fibebocai/nltk_data/")
import en_core_web_sm
from nltk.stem import WordNetLemmatizer

import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
    
import ray

2022-06-22 16:41:11.828818: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/lsf_10.1.0/10.1/linux3.10-glibc2.17-x86_64/lib:/home/fibebocai/virtualenvs/fibebocai_text/lib/python3.7/site-packages/pyoracleclient/instantclient
2022-06-22 16:41:11.828839: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<font size =5 color="red">Data Processing</font>

In [2]:
"""The oop based class which is responsible for all the data preprocessing needed. If you want to edit more, please find the class
in the APIs folder"""
from APIs.Data_Processing import Data_Processing
from nltk.corpus import stopwords
ticketname = "Data/tickets (6).xlsx"
stopwords = stopwords.words("english")

"""if you find some words which is causing noise in the topic creation or which are not contributing much in the topic 
generation then add the word below"""

stopwords.extend(['from', 'subject', 're', 'edu', 'use', 'infineon', 'camstar','com', 'screenshot','yoda','lot','iptester',\
                        "mkz","mkzsapa","mkzsapaxx","site","xml", "total","camstarportal","camstarweb","team","go","gk","gl","alf",\
                        "give","zhouyang","morning","get","getquerytext","ghani","unknown","glue","generate","generation","green",\
                        "afternoon","good","evening","night","user","freeze","full","forward","hello","baby","id", "cannot","unable","add",\
                        "flex", "error message", "error","able","due","mgt","sba","allow","bin","still","workflow","hence","ok","since","critical",\
                        "urgent", "priority", "request","need", "main", "create","would","like","track","must",\
                        "prior","high","find","type","select", "job","fa","type","show", "status",\
                        "move", "contact", "affected","area", "number","server","bth","info","sin"])
a = Data_Processing(ticketname, stopwords) 
[df_reviews, data_words] = a.getdata()

/home/fibebocai/virtualenvs/fibebocai_text/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


<font size=5 color="red">sklearn LDA Model</font>

<font size=3 color="blue"> Library Imports for the model</font>

In [3]:
""" Libraries relevant to the model only"""
import numpy as np
from sklearn import decomposition
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
import gensim.corpora as corpora
import pandas as pd

"""The oop based class which I created for the purpose of training and testing of the LDA model of sklearn"""
from APIs.sklearnLDA import sklearnLDA_NMF


<font size=3 color="blue">Hyperparameters</font>

In [14]:
num_topics = 20
num_words = 15
learning_decay = 0.9
alpha = 0.001
eta = "default"
learning_method = 'batch'
batch_size = 1028
learning_offset = 10
n_top_words = 12
epochs = 200
random_state = 50
ngram_range =(1,3)
max_df = 0.9
min_df = "default"
max_features = 500
lda_topics = [str(i) for i in range(num_topics)]

""" for saving the hyperparameters"""

h_list = [num_topics, num_words, learning_decay, alpha, eta, learning_method, batch_size, learning_offset,\
         epochs, random_state, ngram_range, max_df, min_df, max_features]
columns = ["num_topics", "num_words","learning_decay", "alpha", "eta", "learning_method",  "batch_size","learning_offset", \
           "epochs",  "random_state", "ngram_range","max_df", "min_df", "max_features","lda","nmf"]
# h_dataframe = pd.DataFrame(np.array(h_list), columns = columns)

<font size=3 color="blue">Model</font>

<font size=3 color = "green">LDA model</font>

In [16]:
title_count = 19

In [17]:
model = "lda"
b = sklearnLDA_NMF.remote(model,df_reviews, learning_decay, alpha, eta, epochs, learning_method, learning_offset, batch_size, num_topics, num_words, random_state, ngram_range,\
         min_df, max_df, max_features)
[lda, vectorizer_cv, X_cv, topics, df_doc_topic, significant_topic] = ray.get(b.topic_gen.remote())
df_topic_table = ray.get(b.table_formatting.remote(df_doc_topic))
df_topic_table.to_excel("Results/df_topic_table_lda_%d.xlsx"%title_count)
ray.get(b.get_model_topics.remote(lda_topics, n_top_words)).to_excel("Topics/topics_lda_%d.xlsx"%title_count)
ray.shutdown()

(sklearnLDA_NMF pid=3412439) /home/fibebocai/virtualenvs/fibebocai_text/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
(sklearnLDA_NMF pid=3412439)   warnings.warn(msg, category=FutureWarning)


<font size=3 color = "green">NMF model</font>

In [18]:
model = "nmf"
b = sklearnLDA_NMF.remote(model,df_reviews, learning_decay, alpha, eta, epochs, learning_method, learning_offset, batch_size, num_topics, num_words, random_state, ngram_range,\
         min_df, max_df, max_features)
[nmf, vectorizer_cv, X_cv, topics, df_doc_topic, significant_topic] = ray.get(b.topic_gen.remote())
df_topic_table = ray.get(b.table_formatting.remote(df_doc_topic))
df_topic_table.to_excel("Results/df_topic_table_nmf_%d.xlsx"%title_count)
ray.get(b.get_model_topics.remote(lda_topics, n_top_words)).to_excel("Topics/topics_nmf_%d.xlsx"%title_count)

2022-06-22 17:09:27,729	WARNING worker.py:917 -- File descriptor limit 4000 is too low for production servers and may result in connection errors. At least 8192 is recommended. --- Fix with 'ulimit -n 8192'
(sklearnLDA_NMF pid=3416470) /home/fibebocai/virtualenvs/fibebocai_text/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
(sklearnLDA_NMF pid=3416470)   FutureWarning,
(sklearnLDA_NMF pid=3416470) /home/fibebocai/virtualenvs/fibebocai_text/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
(sklearnLDA_NMF pid=3416470)   ConvergenceWarning,
(sklearnLDA_NMF pid=3416470) /home/fibebocai/virtualenvs/fibebocai_text/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names i

<font size=3 color = "green">Saving the Hyperparamters</font>

In [19]:
import csv
h_list.append("topic_lda_%d"%title_count)
h_list.append("topic_nmf_%d"%title_count)
f = open('hyperparameters.csv', 'a', newline='')
writer= csv.writer(f)
# writer.writerow(columns)
writer.writerow(h_list)
f.close()

<font size=3 color="blue">Testing With The Subcategories</font>

In [ ]:
df_test = pd.read_excel("Data/test_data_jcbe.xlsx")
df_test["topics"] = df_test.desc.map(lambda x:b.get_inference(lda_topics, x,0))
df_test.to_excel("Data/test_result.xlsx")
count = 0
for i in range(df_test.shape[0]):
    if df_test['topics'].loc[i]=="2" or df_test['topics'].loc[i]=='1':
        count+=1
count/df_test.shape[0]*100

In [ ]:
df_test = pd.read_excel("Data/eaf_test.xlsx")
df_test["topics"] = df_test.desc.map(lambda x:b.get_inference(lda_topics, x,0))
df_test.to_excel("Data/test_result_eaf.xlsx")

In [ ]:
count = 0
for i in range(df_test.shape[0]):
    if df_test['topics'].loc[i]=="5" or df_test['topics'].loc[i]=='10' or df_test['topics'].loc[i]=='3':
        count+=1
count/df_test.shape[0]*100

<font size =5 color="red">Cluster Transformer</font>

In [8]:
import ClusterTransformer.ClusterTransformer as ctrans
import pandas as pd

cr = ctrans.ClusterTransformer()
model_name = "allenai/longformer-base-4096"

<font size=3 color="blue">Hyperparameters</font>

In [ ]:
batch_size = 2
max_seq_length = 64
convert_to_numpy = False
normalize_embeddings = False
neighborhood_min_size = 1
cutoff_threshold = 0.83
kmeans_max_iter = 100
kmeans_random_state = 42
kmeans_no_cluster = 6
li_sentence = df_reviews['Desc'][:100]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

<font size=3 color="blue">Model</font>

In [ ]:
embeddings = cr.model_inference(li_sentence, batch_size, model_name, max_seq_length, normalize_embeddings, convert_to_numpy)
output_dict = cr.neighborhood_detection(li_sentence, embeddings, cutoff_threshold, neighborhood_min_size)
output_kmeans_dict = cr.kmeans_detection(li_sentence, embeddings, kmeans_no_cluster, kmeans_max_iter, kmeans_random_state)
neighborhood_detection_df = cr.convert_to_df(output_dict)
kmeans_df = cr.convert_to_df(output_kmeans_dict)
print(f'DataFrame from neighborhood detection:\n {neighborhood_detection_df}')
print(f'DataFrame from Kmeans detection:\n {kmeans_df}')
cr.plot_cluster(neighborhood_detection_df)
cr.plot_cluster(kmeans_df)